Данный материал основан на статье [A Quick Introduction to Neural Networks](https://ujjwalkarn.me/2016/08/09/quick-intro-neural-networks/)

# AI Community @ Семинар № 6
## Обратное распространение ошибки (Backward Propagation)

Мы вывели формулу для одного нейрона, но что делать, если количество слоёв достаточно велико? Бесконечно считать производные правилом цепи? Ответ - back-propagation.  
**Backward Propagation** - самый популярный метод обучения нейронных сетей. Он подразумевает, что после прямого прохода данных через сеть (forward этап), часть информации, необходимая для обновления весов нейронов, сохраняется и используется при обратном проходе через сеть (backward этап).  
В общем, алгоритм выглядит так:
1. Проход данных через сеть в прямом направлении
2. Полученный результат сравнивается с известными значениями из выборки.
3. Ошибка распространяется обратно по сети, обновляя веса нейронов так. В следующий раз сеть "должна" ошибаться меньше.

![Минимизация функции потерь](images_old/nn_cost.png)
[Источник картинки](https://github.com/rasbt/python-machine-learning-book/blob/master/faq/visual-backpropagation.md)

#### Обратное распространение ошибки на примере

![Оценки студентов](images_old/students_eval_table.png)

Сеть выдала неправильный вывод на примере из первой строки таблицы:
![Некорректный вывод сети](images_old/incorrect_nn_output.png)

Хорошо, обновим веса согласно ошибке:
![Обновление весов](images_old/weights_updating.png)

Теперь сеть более точна в своих предсказаниях:
![Скорректированный выход сети](images_old/corrected_nn_output.png)

#### Пример вывода градиентов в back-propagation

[Источник](https://www.ics.uci.edu/~pjsadows/notes.pdf)  
Теперь давайте сделаем вывод градиентов на конктреном примере. Пусть нашей функцией ошибки будет функция *cross-entropy*:
$$E = - \sum_{i=1}^{nout}{t_i log(y_i) + (1-t_i)log(1-y_i)}$$

Функцией активации в нейронах последнего слоя будет сигмоид:
$$y_i = \frac{1}{1+e^{-s_i}} \text{, where } s_i = \sum_{j}{h_j \cdot w_{ji}}$$

Для обновления весов между предпоследним и последним слоями нам нужно посчитать градиент функции ошибки по переменным весов между этими слоями:
$$\frac{\partial E}{\partial w_{ji}} = \frac{\partial E}{\partial y_i} \frac{\partial y_i}{\partial s_i} \frac{\partial s_i}{\partial w_{ji}}$$

Найдем требуемые производные:
$$\frac{\partial E}{\partial y_i} = \frac{-t_i}{y_i} + \frac{1-t_i}{1-y_i} = \frac{y_i-t_i}{y_i(1-y_i)}$$
$$\frac{\partial y_i}{\partial s_i} = y_i(1-y_i)$$
$$\frac{\partial s_i}{\partial w_{ji}} = h_j$$

Теперь, когда соберем все вместе:
$$\frac{\partial E}{\partial w_{ji}} = (y_i-t_i)h_j$$

Все, мы получили градиенты весов между двумя последними слоями. Повторив то же самое для весов между 1-м и 2-м слоями и считая, что функцией активации во 2-м слое был так же сигмоид $h_j=\frac{1}{1+e^{-s_j^1}}$, мы получим следующие градиенты весов между 1-м и 2-м слоями:
$$\frac{\partial E}{\partial w_{kj}^1} = \frac{\partial E}{\partial s_{j}^1} \frac{\partial s_{j}^1}{\partial w_{kj}} = \sum_{i=1}^{nout}{(y_i-t_i)(w_{ji})(h_j(1-h_j))(x_k)}$$

### Пример обучения двуслойного перцептрона на MNIST

In [ ]:
import torch
from torchvision.datasets import MNIST
from torch import nn
from torch.optim import SGD
from torchvision.transforms import Normalize
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchsummary import summary
from tqdm import tqdm_notebook as tqdm

# Кол-во примеров, которые будут проходить через сеть при одном проходе
batch_size = 128
num_classes = 10
epochs = 20

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# Загрузка и предобработка данных

# Загружаем датасеты для тренировки и теста
MNIST('data', train=True, download=True)
MNIST('data', train=False, download=True)

class MNISTDataset(Dataset):
    def __init__(self, data_path):
        data = torch.load(data_path)
        self.data = data[0].reshape(-1, 28 * 28).float() / 255
        self.label = data[1]

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, index):
        image = self.data[index]
        label = self.label[index]
        return image, label
    
train = MNISTDataset('data/MNIST/processed/training.pt')
test = MNISTDataset('data/MNIST/processed/test.pt')

# Исходные данные представляют изображения и класс-цифра как метки.
print(f'Train data shape: {list(train.data.shape)},', f'Train labels shape: {list(train.label.shape)}')
print(f'Test data shape: {list(test.data.shape)},', f'Test labels shape: {list(test.label.shape)}')

In [ ]:
# Создаем нашу сеть!

class Model(nn.Module):
    """Simple NN with hidden layers [300, 100]
    """
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 300, bias=True)
        self.fc2 = nn.Linear(300, 100, bias=True)
        self.fc3 = nn.Linear(100, 10, bias=True)
        
        # Этот блок достаточно создать один раз, так как он не содержит обучаемых параметров
        self.relu = nn.ReLU()

    def forward(self, x):
        x1 = self.relu(self.fc1(x))
        x2 = self.relu(self.fc2(x1))
        x3 = self.fc3(x2)
        return x3
    
model = Model().to(device)
summary(model, (1, 28 * 28), device=device)

In [ ]:
# Устанавливаем функцию ошибки и метод оптимизации модели

criterion = nn.CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=0.03)

# Вместо того, что бы самим разбивать данные на батчи, мы предоставим эту работу pytorch

train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False)

In [ ]:
# Обучаем нашу модель и тестируем точность классификации цифр

for epoch in range(epochs):
    model = model.train()
    t = tqdm(train_loader, total=len(train_loader))
    for data, label in t:
        data = data.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, label)
        
        loss.backward()
        optimizer.step()
        t.set_description_str(desc=f'Loss={float(loss.data):.3f}', refresh=False)
        
predicted_labels = []
true_labels = []
model = model.eval()
for data, label in test_loader:
    data = data.to(device)
    
    output = model(data)
    predicted_labels.append(output.argmax(dim=1).cpu())
    true_labels.append(label)
    
predicted_labels = torch.cat(predicted_labels)
true_labels = torch.cat(true_labels)
print("Test accuracy:", (predicted_labels == true_labels).float().mean())

Что дальше? Пробуйте [новые слои](https://keras.io/layers/core/), практикуйтесь на других простых примерах.  
Посмотрите [другие функции активации](http://ruder.io/optimizing-gradient-descent/index.html#rmsprop)